# User interface

In [30]:
from nilearn.plotting import plot_anat, show
import ipywidgets as widgets
import utils
import os
from importlib import reload
from nilearn import plotting
from nilearn import image as nli
reload(utils)
import nibabel as nib
import numpy as np
from ipywidgets import HBox, VBox
import numpy as np
from nilearn.plotting import plot_anat, show
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output


## Functions

### Misc functions

In [54]:


def display_markdown():
    # This function displays the markdown text, project_dict is 
    # a dictionary containing the project information
    markdown_text = f"""
    ### Experiment Data
    **User:** {project_dict['User']}  \n
    **Dataset:** {project_dict['Dataset']}  \n
    **Session:** {project_dict['Session']}  \n
    **Task:** {project_dict['Task']}  \n
    **Participants:** {project_dict['Participants']}  \n
    **Runs:** {project_dict['Runs']}  \n
    **Specie:** {project_dict['Specie']}  
    """

def create_input_widgets():
    input_widgets = {
        "User": widgets.Text(value='raulh87', description='User:'),
        "Dataset": widgets.Text(value='Complex', description='Dataset:'),
        "Session": widgets.Text(value='', description='Session (optional):'),
        "Task": widgets.Text(value='visual', description='Task:'),
        "Participants": widgets.Text(value='1,2,3,4,5', description='Participant numbers:'),
        "Runs": widgets.Text(value='1,2,3', description='Runs available (1,2,3,...)'),
        "Specie": widgets.Text(value='D', description='Specie (D or H):'),
    }
    return input_widgets

def update_data(b):
    global project_dict
    #clear_output(wait=True)
    for key, widget in input_widgets.items():
        if key == 'Participants' or key == 'Runs':
            project_dict[key] = [int(i) for i in widget.value.split(',')]
        else:
            project_dict[key] = widget.value

    # update the project_dict in the screen
    update_project_dict()

def update_project_dict():
    row_2.clear_output()
    for key, value in project_dict.items():
        #print(f"{key}: {value}")
        with row_2:
            print(f"{key}: {value}")


def cancel_update(b):
    #clear_output(wait=True)
    for key, widget in input_widgets.items():
        widget.value = project_dict[key]
    
    # update the project_dict in the screen
    update_project_dict()

# # Define the class named Process
# class Process:
#     def __init__(self, name, by_run, process_used):
#         self.name = name
#         self.by_run = by_run
#         self.process_used = process_used


# process_list is a list of Process objects
# process_list = [
#     Process('Preprocess', True, 'preprocess_run'),
#     Process('Mean fct', False, 'get_mean_fct'),
#     Process('Crop', False, 'crop_interface'),
#     Process('BET', False, 'bet_interface'),
#     Process('Mean to atlas', False, 'mean_to_STD'),
#     Process('Runs to atlas', True, 'run_to_STD'),
#     Process('Motion', False, 'process_motion'),
# ]

# process_list as dictionary process_dict
process_dict = {
    'preprocess_run': {'name': 'Preprocess', 'by_run': True, 'process_used': 'preprocess_run'},
    'get_mean_fct': {'name': 'Mean fct', 'by_run': False, 'process_used': 'get_mean_fct'},
    'crop_interface': {'name': 'Crop', 'by_run': False, 'process_used': 'crop_interface'},
    'bet_interface': {'name': 'BET', 'by_run': False, 'process_used': 'bet_interface'},
    'mean_to_STD': {'name': 'Mean to atlas', 'by_run': False, 'process_used': 'mean_to_STD'},
    'run_to_STD': {'name': 'Runs to atlas', 'by_run': True, 'process_used': 'run_to_STD'},
    'process_motion': {'name': 'Motion', 'by_run': False, 'process_used': 'process_motion'},
}

process_name_list = [process[1]['name'] for process in process_dict.items()]

### Preprocessing functions

In [86]:
# Define the dictionary and the display functions
project_dict = {
    "User": "",
    "Dataset": "",
    "Session": "",
    "Task": "",
    "Participants": "",
    "Runs": "",
    "Specie": "",
}
project_dict = {
    "User": "raulh87",
    "Dataset": "Complex",
    "Session": "",
    "Task": "visual",
    "Participants": [1,2,3,4,5],
    "Runs": [1,2,3],
    "Specie": "D",
}
# Create a list to store the jobs
job_list = []

# Create the input widgets
input_widgets = create_input_widgets()
update_button = widgets.Button(description="Update")
cancel_button = widgets.Button(description="Cancel")

# Assign an event to the button
update_button.on_click(update_data)
cancel_button.on_click(cancel_update)

# Create a tab widget
tab = widgets.Tab()

# Create the rows to store the widgets
row_1 = widgets.VBox([widgets.VBox(list(input_widgets.values())), update_button, cancel_button])
row_2 = widgets.Output(layout={'border': '1px solid black'})                     

# Create the columns to store in tab 1
data_entry_tab = widgets.HBox([row_1,row_2])

#participant_list = [f"{project_dict['Specie']}-sub-{i:03d}" for i in project_dict['Participants']]

# Create the sub_selector dropdown for participants
sub_selector = widgets.Dropdown(
    options=[f"{project_dict['Specie']}-sub-{i:03d}" for i in project_dict['Participants']],
    value=f"{project_dict['Specie']}-sub-{project_dict['Participants'][0]:03d}",
    description='Participants:',
    disabled=False,
)

# Create the process_selector ToggleButtons for process based on process_list
process_selector = widgets.ToggleButtons(
    options=[process[1]['name'] for process in process_dict.items()],
    description='Process:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    # icons=['check'] * 3
)

# Create checkbox to select the runs to process
run_selector = widgets.HBox(
    [widgets.Checkbox(value=True, description=f"Run {i}") for i in project_dict['Runs']]
)


# Button to schedule the process, make the button black with white text
run_button = widgets.Button(
    description='Add job',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Add selected process to the job list   ',
    icon='book',
)

run_jobs_button = widgets.Button(
    description='Run jobs',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run the selected jobs',
    icon='mail-forward'
)

# Create job_dict to store the process information
job_dict = {
    "User": project_dict['User'],
    "Dataset": project_dict['Dataset'],
    "Session": project_dict['Session'],
    "Task": project_dict['Task'],
    "sub_N":  int(sub_selector.value.split('-')[-1]),
    "run_N": [i+1 for i, run in enumerate(run_selector.children) if run.value],
    "Specie": project_dict['Specie'],
    "Process": process_selector.value,
    "Status": "Scheduled",
}

def update_sub(b):
    global job_dict
    job_dict['sub_N'] = int(sub_selector.value.split('-')[-1])

def process_selector_button_pressed(b):
    global job_dict
    # get the key of the process selected matching process_selector.value with process_name_list
    # process_key = [key for key, value in process_dict.items() if value['name'] == job_dict['Process']][0]
    process_key = [key for key, value in process_dict.items() if value['name'] == process_selector.value][0]

    job_dict['Process'] = process_selector.value
    #print(job_dict['Process'])
    # check if the process is by run
    if process_dict[process_key]['by_run']:
        run_selector.children = [widgets.Checkbox(value=True, description=f"Run {i}") for i in project_dict['Runs']]
    else:
        run_selector.children = [widgets.Checkbox(value=True, description=f"Run {i}", disabled=True) for i in project_dict['Runs']]
    # update run_selector
    #print(process_key)
    


def update_runs(b):
    global job_dict
    job_dict['run_N'] = [i+1 for i, run in enumerate(run_selector.children) if run.value]
    #print(job_dict['run_N'])

# Create function for run_button
def add_job_button_pressed(b):
    global job_dict, job_list
    # clear the output
    #clear_output(wait=True)
    #Output.clear_output()
        # print the job_dict

    # add the job_dict to the job_list
    job_list.append(job_dict.copy())
    print('job added number of jobs: ' + str(len(job_list)))
    # for key, value in job_dict.items():
    #     # print the key and value and add a line break
    #     print(f"{key}: {value}")

def run_jobs(b):
    global job_list
    for job in job_list:
        print(job)
        #utils.run_process(job)
        
    
# add function to sub_selector
sub_selector.observe(update_sub, names='value')

# add function to process_selector
process_selector.observe(process_selector_button_pressed, names='value')

# add function to run_selector
for run in run_selector.children:
    run.observe(update_runs, names='value')

# attach function to run_button
run_button.on_click(add_job_button_pressed)

# attach function to run_jobs_button
run_jobs_button.on_click(run_jobs)

# Create Scheduler tab
scheduler_tab = widgets.VBox([sub_selector,process_selector,run_selector,run_button,run_jobs_button])

# Organize contents into tabs
tab_contents = ['Data Entry', 'Scheduler', 'Tab 3']
tab_list = [scheduler_tab,
            data_entry_tab,
            widgets.Text(description="Tab 3 Content")]
tab.children = tab_list

for i, name in enumerate(tab_contents):
    tab.set_title(i, name)



# Display the tab widget
display(tab)




In [72]:
for key, value in process_dict.items():
    print(f"key: {key}: value: {value}")

key: preprocess_run: value: {'name': 'Preprocess', 'by_run': True, 'process_used': 'preprocess_run'}
key: get_mean_fct: value: {'name': 'Mean fct', 'by_run': False, 'process_used': 'get_mean_fct'}
key: crop_interface: value: {'name': 'Crop', 'by_run': False, 'process_used': 'crop_interface'}
key: bet_interface: value: {'name': 'BET', 'by_run': False, 'process_used': 'bet_interface'}
key: mean_to_STD: value: {'name': 'Mean to atlas', 'by_run': False, 'process_used': 'mean_to_STD'}
key: run_to_STD: value: {'name': 'Runs to atlas', 'by_run': True, 'process_used': 'run_to_STD'}
key: process_motion: value: {'name': 'Motion', 'by_run': False, 'process_used': 'process_motion'}


In [ ]:
process_key = [key for key, value in process_dict.items() if value['name'] == job_dict['Process']][0]


In [77]:
for key, value in process_dict.items():
    if value['name'] == job_dict['Process']:
        process_key = key
process_key

'mean_to_STD'

In [79]:
for key, value in process_dict.items():
    if value['name'] == process_selector.value:
        process_key = key

process_key = [key for key, value in process_dict.items() if value['name'] == process_selector.value][0]


Preprocess
Mean fct
Crop
BET
Mean to atlas
Runs to atlas
Motion


In [ ]:
process_dict

In [71]:
[key for key, value in process_dict.items() if value['name'] == job_dict['Process']]

[]

In [70]:
process_key

'get_mean_fct'

In [ ]:
process_key = [key for key, value in process_dict.items() if value['name'] == job_dict['Process']][0]

In [65]:
job_dict

{'User': 'raulh87',
 'Dataset': 'Complex',
 'Session': '',
 'Task': 'visual',
 'sub_N': 1,
 'run_N': [1, 2, 3],
 'Specie': 'D',
 'Process': 'preprocess_run',
 'Status': 'Scheduled'}

In [67]:
process_dict.items()

dict_items([('preprocess_run', {'name': 'Preprocess', 'by_run': True, 'process_used': 'preprocess_run'}), ('get_mean_fct', {'name': 'Mean fct', 'by_run': False, 'process_used': 'get_mean_fct'}), ('crop_interface', {'name': 'Crop', 'by_run': False, 'process_used': 'crop_interface'}), ('bet_interface', {'name': 'BET', 'by_run': False, 'process_used': 'bet_interface'}), ('mean_to_STD', {'name': 'Mean to atlas', 'by_run': False, 'process_used': 'mean_to_STD'}), ('run_to_STD', {'name': 'Runs to atlas', 'by_run': True, 'process_used': 'run_to_STD'}), ('process_motion', {'name': 'Motion', 'by_run': False, 'process_used': 'process_motion'})])

In [66]:
process_key = [key for key, value in process_dict.items() if value['name'] == job_dict['Process']][0]
process_key

IndexError: list index out of range

In [60]:
process_name_list = [process[1]['name'] for process in process_dict.items()]
# Check which process matches between process_name_list and job_dict['Process']


process_key

'get_mean_fct'

In [62]:
process_dict[process_key]['by_run']

False

In [61]:
process_dict

{'preprocess_run': {'name': 'Preprocess',
  'by_run': True,
  'process_used': 'preprocess_run'},
 'get_mean_fct': {'name': 'Mean fct',
  'by_run': False,
  'process_used': 'get_mean_fct'},
 'crop_interface': {'name': 'Crop',
  'by_run': False,
  'process_used': 'crop_interface'},
 'bet_interface': {'name': 'BET',
  'by_run': False,
  'process_used': 'bet_interface'},
 'mean_to_STD': {'name': 'Mean to atlas',
  'by_run': False,
  'process_used': 'mean_to_STD'},
 'run_to_STD': {'name': 'Runs to atlas',
  'by_run': True,
  'process_used': 'run_to_STD'},
 'process_motion': {'name': 'Motion',
  'by_run': False,
  'process_used': 'process_motion'}}

'Mean fct'

In [6]:
options=[f"Run {i}" for i in project_dict['Runs']]
options

['Run 1', 'Run 2', 'Run 3']

In [53]:
process_dict

{'preprocess_run': {'name': 'Preprocess',
  'by_run': True,
  'process_used': 'preprocess_run'},
 'get_mean_fct': {'name': 'Mean fct',
  'by_run': False,
  'process_used': 'get_mean_fct'},
 'crop_interface': {'name': 'Crop',
  'by_run': False,
  'process_used': 'crop_interface'},
 'bet_interface': {'name': 'BET',
  'by_run': False,
  'process_used': 'bet_interface'},
 'mean_to_STD': {'name': 'Mean to atlas',
  'by_run': False,
  'process_used': 'mean_to_STD'},
 'run_to_STD': {'name': 'Runs to atlas',
  'by_run': True,
  'process_used': 'run_to_STD'},
 'process_motion': {'name': 'Motion',
  'by_run': False,
  'process_used': 'process_motion'}}

In [ ]:
# Create the process_menu ToggleButtons for process based on process_list
process_menu = widgets.ToggleButtons(
    options=[process_dict for process in process_list],
    description='Process:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    # icons=['check'] * 3
)



In [47]:
for process in process_dict.items():
    print(process[1]['name'])

options = [process[1]['name'] for process in process_dict.items()]
print(options)

Preprocess
Mean fct
Crop
BET
Mean to atlas
Runs to atlas
Motion
['Preprocess', 'Mean fct', 'Crop', 'BET', 'Mean to atlas', 'Runs to atlas', 'Motion']


In [45]:
process[1]['name']

'Preprocess'

In [6]:
f"{project_dict['Specie']}-sub-{i:03d}" for i in project_dict['Participants']

SyntaxError: invalid syntax (4281251277.py, line 1)

In [25]:
project_dict['Participants']

# Create a list of participants padded with three zeros
participant_list = [f"{project_dict['Specie']}-sub-{i:03d}" for i in project_dict['Participants']]
print(participant_list)

['D-sub-001', 'D-sub-002', 'D-sub-003', 'D-sub-004', 'D-sub-005']


In [11]:
# Transform project_dict['Participants'] into a list of integers:
participants_list = project_dict['Participants'].split(',')
participants_list = [int(i) for i in participants_list]
participants_list



[1, 2, 3, 4, 5]

## Scheduler

In [5]:
# Define the class named Process
class Process:
    def __init__(self, name, by_run, process_used):
        self.name = name
        self.by_run = by_run
        self.process_used = process_used

process_list = [
    Process('Preprocess', True, 'preprocess_run'),
    Process('Mean fct', False, 'get_mean_fct'),
    Process('Crop', False, 'crop_interface'),
    Process('BET', False, 'bet_interface'),
    Process('Mean to atlas', False, 'mean_to_STD'),
    Process('Runs to atlas', True, 'run_to_STD'),
    Process('Motion', False, 'process_motion'),
]

runs_available = [1,2,3,4]
subs_possible = [1,2,3,4]

process_list[0].name


'Preprocess'

In [ ]:
# Define a new class Job
class Job:
    def __init__(self, process, participant, run):
        self.process = process
        self.participant = participant
        self.run = run
        self.status = 'Pending'
        

In [12]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Define the class named Process
class Process:
    def __init__(self, name, by_run, process_used):
        self.name = name
        self.by_run = by_run
        self.process_used = process_used

# Create a list of processes
process_list = [
    Process('Preprocess', True, 'preprocess_run'),
    Process('Mean fct', False, 'get_mean_fct'),
    Process('Crop', False, 'crop_interface'),
    Process('BET', False, 'bet_interface'),
    Process('Mean to atlas', False, 'mean_to_STD'),
    Process('Runs to atlas', True, 'run_to_STD'),
    Process('Motion', False, 'process_motion'),
]

subs_possible = [1, 2, 3, 4]
runs_available = [1, 2, 3, 4]



GridBox(children=(Label(value=''), Label(value='Preprocess'), Label(value='Mean fct'), Label(value='Crop'), La…

In [ ]:
preprocess = {
    'name': 'Preprocess',
    'by_run': False,
    'function': 'preprocess_run'
    }




In [4]:
import ipywidgets as widgets
from IPython.display import display

def create_button_matrix(rows, columns):
    # This function creates a button and assigns an on_click event
    def create_button(row, col):
        button = widgets.Button(description=f"Button {row+1},{col+1}")
        button.on_click(lambda b: print(f"Button at Row {row+1}, Column {col+1} was pressed"))
        return button

    # Create a grid of buttons
    grid = widgets.GridBox(children=[
        create_button(row, col) for row in range(rows) for col in range(columns)
    ], layout=widgets.Layout(grid_template_columns="repeat(4, 100px)"))
    
    display(grid)

# Create a 3x4 matrix of buttons
create_button_matrix(3, 4)


GridBox(children=(Button(description='Button 1,1', style=ButtonStyle()), Button(description='Button 1,2', styl…

Button at Row 1, Column 1 was pressed
Button at Row 1, Column 2 was pressed
Button at Row 1, Column 3 was pressed
Button at Row 1, Column 4 was pressed
Button at Row 2, Column 1 was pressed
Button at Row 2, Column 2 was pressed
Button at Row 2, Column 3 was pressed
Button at Row 2, Column 4 was pressed
Button at Row 3, Column 1 was pressed
Button at Row 3, Column 2 was pressed
Button at Row 3, Column 3 was pressed
Button at Row 3, Column 4 was pressed
Button at Row 2, Column 4 was pressed
Button at Row 1, Column 4 was pressed


In [6]:
# Create a tab widget
tab = widgets.Tab()

# Create contents for each tab
children = [widgets.Text(description=f"Tab {i+1}") for i in range(3)]

# Set the children for the tab widget
tab.children = children

# Name your tabs
for i in range(len(children)):
    tab.set_title(i, f'Tab {i+1}')

# Display the tab widget
display(tab)
